In [ ]:
 !nvidia-smi

Mon Mar 27 18:15:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pwd

'/content'

In [ ]:
!unzip /content/drive/MyDrive/Images_ImageNet_and_Scrapper_Resized_Split.zip

Streaming output truncated to the last 5000 lines.
  inflating: Images_ImageNet_and_Scrapper_Resized_Split/train/banana/n07753592_8860_object_0_resized_200x200.png  
  inflating: Images_ImageNet_and_Scrapper_Resized_Split/train/banana/n07753592_8891_object_0_resized_200x200.png  
  inflating: Images_ImageNet_and_Scrapper_Resized_Split/train/banana/n07753592_8916_object_0_resized_200x200.png  
  inflating: Images_ImageNet_and_Scrapper_Resized_Split/train/banana/n07753592_8916_object_1_resized_200x200.png  
  inflating: Images_ImageNet_and_Scrapper_Resized_Split/train/banana/n07753592_8919_object_0_resized_200x200.png  
  inflating: Images_ImageNet_and_Scrapper_Resized_Split/train/banana/n07753592_8919_object_1_resized_200x200.png  
  inflating: Images_ImageNet_and_Scrapper_Resized_Split/train/banana/n07753592_891_object_0_resized_200x200.png  
  inflating: Images_ImageNet_and_Scrapper_Resized_Split/train/banana/n07753592_891_object_1_resized_200x200.png  
  inflating: Images_ImageNet_an

In [ ]:
cd /content/drive/MyDrive/uois/resnet

/content/drive/MyDrive/uois/resnet


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets

from tqdm import tqdm

In [ ]:
# Step 1: Load the pre-trained ResNet model
model = models.resnet152(pretrained=True)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

In [ ]:
# Step 2: Replace the first layer kernel size or smaller images and last fully connected layer with a new layer that matches the number of classes in your dataset

# Modify the first layer to accept 32x32 input images
# model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
# model.maxpool = nn.Identity()

num_classes = 8 # banana, bottle, bowl, box, cup, orange, wallet, other
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

In [ ]:
# Step 3: Load your dataset and prepare it for training
data_transforms = transforms.Compose([
    transforms.Resize(200),  # Resize the image 
    # transforms.CenterCrop(224),  # Crop the image to 224x224, centered
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize the image with ImageNet mean and std
])

# data_transforms = transforms.Compose([
#     transforms.Resize(32),
#     transforms.RandomHorizontalFlip(p=0.5),
#     transforms.RandomRotation(degrees=45),
#     transforms.CenterCrop(224),
#     transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
#     transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# ])


train_dataset = datasets.ImageFolder('/content/Images_ImageNet_and_Scrapper_Resized_Split/train', transform=data_transforms)  # Load the training dataset
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)  # Create a DataLoader for the training dataset

val_dataset = datasets.ImageFolder('/content/Images_ImageNet_and_Scrapper_Resized_Split/val', transform=data_transforms)  # Load the validation dataset
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)  # Create a DataLoader for the validation dataset

In [ ]:
# Step 4: Define your loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()  # Use cross-entropy loss
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  # Use stochastic gradient descent optimizer with a learning rate of 0.001 and momentum of 0.9


In [ ]:
# Step 5: Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Use the GPU if available
model.to(device)

for epoch in range(10):  # Train for 10 epochs
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)  # Move inputs and labels to the GPU
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update the parameters
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_dataset)  # Compute the epoch loss
    print('Epoch {} loss: {:.4f}'.format(epoch, epoch_loss))
    # Export the model weights
    torch.save(model.state_dict(), 'model_weights_imagenet_152_scrapped_200'+str(epoch) +'.pth')

100%|██████████| 72/72 [01:41<00:00,  1.41s/it]


Epoch 0 loss: 0.7696


100%|██████████| 72/72 [01:42<00:00,  1.42s/it]


Epoch 1 loss: 0.1416


100%|██████████| 72/72 [01:42<00:00,  1.43s/it]


...

Epoch 7 loss: 0.0140


100%|██████████| 72/72 [01:42<00:00,  1.43s/it]


Epoch 8 loss: 0.0132


100%|██████████| 72/72 [01:42<00:00,  1.43s/it]


Epoch 9 loss: 0.0100


In [ ]:
# Step 6: Evaluate the model on the validation set
model.eval()  # Set the model to evaluation mode
val_acc = 0.0
with torch.no_grad():
    for inputs, labels in tqdm(val_loader):
        inputs, labels = inputs.to(device), labels.to(device)  # Move inputs and labels to the GPU
        outputs = model(inputs)  # Forward pass
        _, preds = torch.max(outputs, 1)  # Get the predicted labels
        val_acc += torch.sum(preds == labels.data)  # Compute the number of correct predictions
val_acc = val_acc / len(val_dataset)  # Compute the validation accuracy
print('Validation accuracy: {:.4f}'.format(val_acc))

100%|██████████| 15/15 [00:05<00:00,  2.71it/s]

Validation accuracy: 0.9847


In [ ]:
# Export the model weights
torch.save(model.state_dict(), 'model_weights_resnet_152_imagenet.pth')